In [1]:
#Author: Sejal Gupta, Twisha Bansal
#References: 
#https://opacus.ai/tutorials/building_image_classifier#Test-the-network-on-test-data
#https://arxiv.org/pdf/1607.00133.pdf

import warnings
warnings.simplefilter("ignore")

MAX_GRAD_NORM = 1.1 #gradient norm bound 'C'
EPSILON = 50.0      #privacy loss, should be small for higher privacy
DELTA = 1e-4        #ε-differential privacy is broken with probability δ (which is preferably smaller than 1/|d|)
EPOCHS = 20
LR = 1e-2           #step size at each iteration

In [2]:
BATCH_SIZE = 64  #logical batch size, which defines how often the model is updated and how much DP noise is added
MAX_PHYSICAL_BATCH_SIZE = 128 #physical batch size, which defines how many samples we process at a time

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

#pre-computed
MNIST_MEAN = (0.1307,)
MNIST_STD_DEV = (0.3081,)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(MNIST_MEAN, MNIST_STD_DEV),
])

In [4]:
from torchvision.datasets import MNIST

DATA_ROOT = '/files/'

#load the MNIST(handwritten digits) dataset
train_dataset = MNIST(
    root=DATA_ROOT, train=True, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
)

test_dataset = MNIST(
    root=DATA_ROOT, train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw



In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
#defining the classification model(convolutional neural network)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [7]:
model = Net()

In [8]:
!pip install opacus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.9/216.9 kB 908.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 24.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cpu
    Uninstalling torch-1.11.0+cpu:
      Successfully uninstalled torch-1.11.0+cpu
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cpu requires torch==1.11.0, but you have torch 1.13.0 which is incompatible.
torchtext 0.12.0 requires torch

In [9]:
from opacus.validators import ModuleValidator

errors = ModuleValidator.validate(model, strict=False)
errors[-5:]

[]

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

model = model.to(device)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.5)

In [12]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [13]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

#sigma depends on epsilon and delta, get the noise scale sigma from the target epsilon and target delta provided
print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

Using sigma=0.3024864196777344 and C=1.1


In [14]:
import numpy as np
from opacus.utils.batch_memory_manager import BatchMemoryManager

#function to train the model
def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []
    
    #To balance peak memory requirement, which is proportional to batch_size^2, and training performance, use BatchMemoryManager
    with BatchMemoryManager(
        data_loader=train_loader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:

        for i, (images, target) in enumerate(memory_safe_data_loader):   
            optimizer.zero_grad()
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # measure accuracy and record loss
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer.step()

            if (i+1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                    f"(ε = {epsilon:.2f}, δ = {DELTA})"
                )

In [15]:
#function to test the model
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

In [16]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model, train_loader, optimizer, epoch + 1, device)

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 2.287196 Acc@1: 13.569050 (ε = 9.66, δ = 0.0001)
	Train Epoch: 1 	Loss: 2.254486 Acc@1: 16.829012 (ε = 11.30, δ = 0.0001)
	Train Epoch: 1 	Loss: 2.211812 Acc@1: 20.121011 (ε = 12.57, δ = 0.0001)
	Train Epoch: 1 	Loss: 2.162817 Acc@1: 22.985064 (ε = 13.64, δ = 0.0001)
	Train Epoch: 2 	Loss: 1.781397 Acc@1: 38.885069 (ε = 15.16, δ = 0.0001)
	Train Epoch: 2 	Loss: 1.730111 Acc@1: 40.813174 (ε = 15.96, δ = 0.0001)
	Train Epoch: 2 	Loss: 1.679309 Acc@1: 42.479926 (ε = 16.71, δ = 0.0001)
	Train Epoch: 2 	Loss: 1.644596 Acc@1: 43.698369 (ε = 17.41, δ = 0.0001)
	Train Epoch: 3 	Loss: 1.434348 Acc@1: 51.482871 (ε = 18.53, δ = 0.0001)
	Train Epoch: 3 	Loss: 1.418786 Acc@1: 52.430807 (ε = 19.15, δ = 0.0001)
	Train Epoch: 3 	Loss: 1.398395 Acc@1: 53.547450 (ε = 19.76, δ = 0.0001)
	Train Epoch: 3 	Loss: 1.384866 Acc@1: 54.536790 (ε = 20.34, δ = 0.0001)
	Train Epoch: 4 	Loss: 1.280211 Acc@1: 60.408074 (ε = 21.28, δ = 0.0001)
	Train Epoch: 4 	Loss: 1.269450 Acc@1: 60.832194 (ε 

In [17]:
top1_acc = test(model, test_loader, device)

	Test set:Loss: 0.529818 Acc: 90.376194 
